In [1]:
# import torch
# import pandas as pd
# import numpy as np 
# from PIL import Image

In [2]:
# inputd = pd.read_pickle("./dataset/file.pkl")
# #lastnp = np.load("file.pkl",allow_pickle=True)
# X_centerIMG =  np.array(inputd["centerimg"].tolist()) # M , width , height
# X_leftIMG = np.array(inputd["leftimg"].tolist())
# X_rightIMG = np.array(inputd["rightimg"].tolist())
# Y_throttle = np.array(inputd["throttle"])
# Y_steering = np.array(inputd["steering"])
# Y_reverse = np.array(inputd["reverse"])
# Y_speed = np.array(inputd["speed"]) #*

# Y = np.vstack((Y_throttle,Y_steering,Y_reverse)).T # M , 3

In [3]:
# inputd.iloc[:,3:].value_counts()


In [4]:
# X_centerIMG.shape

In [5]:
# Y.shape

In [6]:
# X_center_tensor =  torch.tensor(X_centerIMG)

In [7]:
# X_center_tensor.shape
# type(X_center_tensor[0])

# After knowing pytorch datasets module

In [2]:
import os 
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset,DataLoader

from torchvision import transforms,utils
from matplotlib import pyplot as plt

In [8]:
class RacingCarsDataset(Dataset):
    def __init__(self,csv_file,transform=None):
        self.frame = pd.read_pickle(csv_file)
        #self.frame.drop(["reverse"],axis= 1)
        self.centerimg =torch.Tensor(np.array(self.frame["centerimg"].tolist(),dtype=np.float32)) # M , width , height
        self.leftimg = torch.Tensor(np.array(self.frame["leftimg"].tolist(),dtype=np.float32))
        self.rightimg = torch.Tensor(np.array(self.frame["rightimg"].tolist(),dtype=np.float32)) 
        Y_throttle = np.array(self.frame["throttle"],dtype=np.float32)
        Y_steering = np.array(self.frame["steering"],dtype=np.float32)
        Y_reverse = np.array(self.frame["reverse"],dtype=np.float32)
        Y_speed = np.array(self.frame["speed"],dtype=np.float32) #*
        self.Y = torch.Tensor(np.vstack((Y_throttle,Y_steering,Y_reverse),dtype=np.float32).T)
        self.transform=transform
    def __len__(self):
        return len(self.frame)
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index=index.tolist()
        centerimg = self.centerimg[index]
        centerimg =centerimg.reshape((*centerimg.shape,1))
        centerimg=np.array(centerimg.tolist())
        leftimg = self.leftimg[index]
        leftimg =leftimg.reshape((*leftimg.shape,1))
        leftimg=np.array(leftimg.tolist())
        rightimg = self.rightimg[index]
        rightimg =rightimg.reshape((*rightimg.shape,1))
        rightimg=np.array(rightimg.tolist())
        outputs=self.Y[index]
        #outputs=outputs.reshape((*outputs.shape,1))
        sample = {"center": centerimg,"left": leftimg,"right": rightimg, "outputs":outputs}
        if self.transform != None:
            sample["center"] = self.transform(sample["center"])
            sample["left"] = self.transform(sample["left"])
            sample["right"] = self.transform(sample["right"])
            #sample["outputs"] = self.transform(outputs)
        return sample 

        


In [ ]:
rd = RacingCarsDataset("./dataset/file.pkl")
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(12,4))
for i,sample in enumerate(rd):
    print(i,sample['center'].shape,sample['left'].shape,sample['right'].shape,sample["outputs"])

    
    plt.sca(axs[i][0])
    plt.imshow(sample['left'],cmap='gray')
    plt.axis('off')
    plt.sca(axs[i][1])
    plt.imshow(sample['center'],cmap='gray')
    plt.axis('off')
    plt.sca(axs[i][2])
    plt.imshow(sample['right'],cmap='gray')
    plt.axis('off')
    if i==3 : 
        plt.show()
        break

In [3]:
class ToTensor(object):
    def __call__(self,sample):
        center,left,right,Y = sample.values()
        print(center.shape,left.shape,right.shape,sep='\n')
        center=center.transpose((2,0,1))
        left=left.transpose((2,0,1))
        right=right.transpose((2,0,1))
        return {"center": torch.Tensor(center),"left": torch.Tensor(left),"right": torch.Tensor(right), "outputs":torch.Tensor(Y)}


In [ ]:
batch_size=12
M = 6828
num_epochs = 10
height=160
width=320
from torchvision import transforms
training_dataset = RacingCarsDataset("./dataset/file.pkl",transform=transforms.Compose([transforms.ToTensor()]))
training_dataset_loader = DataLoader(training_dataset,batch_size=batch_size,shuffle=True)


In [8]:
# mark

In [5]:
import torch.nn as nn
import torch.functional as F
input_channel=1
hidden_channel=64
output_channel=128
learning_rate =0.01
class Model(nn.Module):

    def __init__(self,input_channels,hidden_channels,output_channels,hidden_layer):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_channels,out_channels=hidden_channels,kernel_size=3) 
        self.conv2 = nn.Conv2d(in_channels=hidden_channels,out_channels=output_channels,kernel_size=3)
        self.bn1 = nn.BatchNorm2d(hidden_channels)
        self.bn2 = nn.BatchNorm2d(output_channels)
        self.fc1 = nn.Linear(379392,hidden_layer)
        self.fc2 = nn.Linear(hidden_layer,3)
        self.max1=nn.MaxPool2d(2,2)
        self.max2=nn.MaxPool2d(2,2)
    def forward(self,input):
        input = self.bn1(self.max1(self.conv1(input)))
        input = self.bn2(self.max2(self.conv2(input)))
        input = input.view(-1, self.num_flat_features(input))
        #print(input.shape)
        input = self.fc1(input)
        input = self.fc2(input)
        return input
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [7]:
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
m = Model(input_channel,hidden_channel,output_channel,125).to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
n_total_steps = len(training_dataset_loader)
for epoch in range(num_epochs):

    running_loss = 0.0

    for i, sample in enumerate(training_dataset_loader):
        centerimg = sample["center"].to(device, dtype=torch.float32)
        labels = sample["outputs"].to(device, dtype=torch.float32)
        #print(centerimg.shape)
        # Forward pass
        outputs = m(centerimg)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()
    if (epoch % 10) == 0:
        torch.save(Model, f'model{epoch/10}.pth')
    print(f'[{epoch + 1}] loss: {running_loss / n_total_steps:.3f}')



[1] loss: 17289.233
[2] loss: 28.763
[3] loss: 120.279
[4] loss: 0.063
[5] loss: 0.106
[6] loss: 190.115
[7] loss: 180863.902
[8] loss: 11.688
[9] loss: 5.705
[10] loss: 3.543


In [18]:
torch.save(Model, 'modellast.pt')

In [6]:
loaded_model = torch.load('./modellast.pt')
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
l = loaded_model(1,64,128,125).to(device)
l.eval()


Model(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=379392, out_features=125, bias=True)
  (fc2): Linear(in_features=125, out_features=3, bias=True)
  (max1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (max2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [11]:

test_dataset = RacingCarsDataset("./dataset/test.pkl",transform=transforms.Compose([transforms.ToTensor()]))
test_dataset_loader = DataLoader(test_dataset,batch_size=12,shuffle=True)
with torch.no_grad():
    n_correct2 = 0
    n_samples = len(test_dataset_loader.dataset)
    MSE=nn.MSELoss()
    for i,sample in enumerate(test_dataset_loader):
        images = sample["center"].to(device,dtype=torch.float32)
        labels = sample["outputs"].to(device,dtype=torch.float32)
        #outputs = model(images)

        # max returns (value ,index)
        #n_correct += (outputs == labels).sum().item()

        outputs2 = l(images)
        n_correct2 +=MSE(outputs2,labels)

    #acc = 100.0 * n_correct / n_samples
    #print(f'Accuracy of the model: {acc} %')

    acc = 100.0 * n_correct2 / n_samples
    print(f'Average loss of the loaded model on saved model: {acc} ')

Average loss of the loaded model on saved model: 2710.649658203125 %
